In [7]:
import pandas as pd

# Read the CSV file into a DataFrame
df = pd.read_csv('DataSet2.csv')

# Generating for the DataFrame
df.describe()


,tokens,tags
count,2807,2807
unique,2806,2770
top,"['I', 'just', 'earned', 'the', ""'"", 'The', 'Da...","['O', 'O', 'O', 'O', 'O', 'B-TVshow', 'I-TVSho..."
freq,2,5


In [45]:
pip install tf2crf

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.6/12.6 MB 50.1 MB/s eta 0:00:0000:0100:01
Note: you may need to restart the kernel to use updated packages.


In [47]:
pip install keras


Note: you may need to restart the kernel to use updated packages.


In [6]:
pip install keras

Note: you may need to restart the kernel to use updated packages.


In [ ]:
POS Tagging

In [80]:
import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
import pandas as pd
import nltk
from nltk import pos_tag
from nltk.tokenize import word_tokenize

# Load the dataset
df = pd.read_csv('DataSet2.csv')

# Function to perform POS tagging on a sentence
def pos_tagging(sentence):
    tokens = word_tokenize(sentence)
    pos_tags = pos_tag(tokens)
    return [tag for _, tag in pos_tags]

# Apply POS tagging to the 'tokens' column
df['pos_tags'] = df['tokens'].apply(lambda x: [pos_tagging(sentence) for sentence in x])
# It assumes that each cell in the 'tokens' column  contains a list of sentences. 
#The list iterates through each sentence in x and applies the pos_tagging function to it.

# Saving the updated dataset to csv file
df.to_csv('DataSet2_with_POS.csv', index=False)


[nltk_data] Downloading package punkt to /Users/anithach/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/anithach/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [ ]:
# CRF model without feature engineering 

In [3]:
import json
import pandas as pd
from sklearn_crfsuite import CRF
from sklearn.model_selection import train_test_split
from sklearn_crfsuite import metrics
from sklearn.metrics import classification_report  # Import directly from sklearn

# Loading the dataset into pandas dataframe
df = pd.read_csv('DataSet2.csv')

# Convert tokens and tags from strings to lists
df['tokens'] = df['tokens'].apply(eval)
df['tags'] = df['tags'].apply(eval)

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(df['tokens'], df['tags'], test_size=0.2, random_state=42)
# Splits the tokens and tags into training and testing sets 
# with 20% of the data reserved for testing.

# Initializing and training the CRF model by using a function from CRF module
crf = CRF()
crf.fit(X_train.tolist(), y_train.tolist())  # Ensuring data is in list format

# Make predictions on test data
y_pred = crf.predict(X_test.tolist())

# Flatten the list of tags for y_test and y_pred to avoid hierachial or multiple tags for a tokens
# and to compute overall metrics
y_test_flat = [tag for sublist in y_test for tag in sublist]
y_pred_flat = [tag for sublist in y_pred for tag in sublist]

# Calculate and print precision, recall, F1-score, and accuracy using functions in metrics module
#The "weighted average" calculates each metric as per its occurence, and is used  in imbalanced datasets where some classes are significantly more common than others.
print("Precision: ", metrics.flat_precision_score(y_test, y_pred, average='weighted', zero_division=0))
print("Recall: ", metrics.flat_recall_score(y_test, y_pred, average='weighted', zero_division=0))
print("F1 Score: ", metrics.flat_f1_score(y_test, y_pred, average='weighted', zero_division=0))
print("Accuracy: ", metrics.flat_accuracy_score(y_test, y_pred))  # Accuracy calculation

report = classification_report(y_test_flat, y_pred_flat)
print(" Classification Report:\n", report) # This prints the parameters for each entity type in the dataset

Precision:  0.7759572716253779
Recall:  0.8384800515236772
F1 Score:  0.8002605362512951
Accuracy:  0.8384800515236772
 Classification Report:
               precision    recall  f1-score   support

    B-TVshow       0.11      0.02      0.03       313
   B-company       0.27      0.09      0.14       329
     B-event       0.22      0.08      0.12       445
     B-group       0.36      0.11      0.17       445
  B-location       0.18      0.02      0.04       265
    B-person       0.55      0.39      0.46       953
   B-product       0.59      0.38      0.46       360
    I-TVShow       0.03      0.01      0.01       434
   I-company       0.04      0.02      0.02       109
     I-event       0.18      0.05      0.07       515
     I-group       0.08      0.01      0.01       159
  I-location       0.00      0.00      0.00       110
    I-person       0.33      0.21      0.25       324
   I-product       0.03      0.01      0.01       137
           O       0.87      0.98      0.92  

In [ ]:
# CRF Model with feature engineering

In [ ]:
#Training dataset

In [10]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn_crfsuite import CRF
from sklearn.metrics import classification_report, accuracy_score
import ast

# Load Data
df = pd.read_csv('DataSet1_with_POS.csv')
df['tokens'] = df['tokens'].apply(ast.literal_eval)
df['tags'] = df['tags'].apply(ast.literal_eval)

def wordstofeatures(sent, i):
# Defining a function to extract features from a word in a sentence.
    word = sent[i] # Storing the word at position i in the sentence.
    features = {
        'bias': 1.0,
        'word.lower()': word.lower(),
        'word[-3:]': word[-3:], # The last three characters of the word.
        'word[-2:]': word[-2:], # The last two characters of the word.
        'word.isupper()': word.isupper(),
        'word.istitle()': word.istitle(),
        'word.isdigit()': word.isdigit(),
    }
    # Contextual features using neighboring words.
    if i > 0:
        features.update({
            '-1:word.lower()': sent[i-1].lower(), # The previous word in lowercase.
            '-1:word.istitle()': sent[i-1].istitle(),
        })
    if i < len(sent) - 1:
        features.update({
            '+1:word.lower()': sent[i+1].lower(),
            '+1:word.istitle()': sent[i+1].istitle(),
        })
    return features

def sent2features(sent):
    return [wordstofeatures(sent, i) for i in range(len(sent))]
# Applying feature extraction to each word in the sentence by using wordstofeatures function.
    # The function iterates over each word by its index,
    # extracts relevant features using the word2features function,
    # and compiles these into a list of feature dictionaries.

X = df['tokens'].apply(sent2features).tolist()
# Converting the tokens to features for all sentences in the dataframe and and converting it to list format for the CRF model.
y = df['tags'].tolist()

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)
# Splitting the dataset into training and testing sets using a 25% data for testing.

crf = CRF(algorithm='lbfgs', c1=0.1, c2=0.1, max_iterations=200, all_possible_transitions=True)
# Initializing the CRF model with the 'lbfgs' optimization algorithm and regularization parameters.

crf.fit(X_train, y_train)

y_pred = crf.predict(X_test)

report = classification_report(y_test, y_pred, output_dict=True)
# Generating a detailed classification report that includes precision, recall, F1-score for each tag.

print(" Classification Report:\n", classification_report(y_test_flat, y_pred_flat))
print("Precision: {:.2f}".format(report['weighted avg']['precision'])) # the format function rounds off the decimal points
print("Recall: {:.2f}".format(report['weighted avg']['recall']))
print("Accuracy: {:.2f}".format(accuracy_score(y_test, y_pred)))
print("F1-score: {:.2f}".format(report['weighted avg']['f1-score']))

 Classification Report:
               precision    recall  f1-score   support

    B-TVshow       0.40      0.22      0.28        55
   B-company       0.33      0.12      0.17        59
     B-event       0.56      0.14      0.22        71
     B-group       0.69      0.37      0.48        59
  B-location       0.58      0.28      0.38        39
    B-person       0.68      0.61      0.64       137
   B-product       0.69      0.46      0.55        63
    I-TVShow       0.51      0.30      0.38        74
   I-company       0.06      0.03      0.04        34
     I-event       0.59      0.17      0.27        58
     I-group       0.43      0.11      0.18        27
  I-location       0.41      0.30      0.35        23
    I-person       0.44      0.53      0.48        40
   I-product       0.00      0.00      0.00        11
           O       0.90      0.98      0.94      3784

    accuracy                           0.87      4534
   macro avg       0.48      0.31      0.36      4534
w

In [ ]:
#Testing Dataset

In [3]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn_crfsuite import CRF
from sklearn.metrics import classification_report, accuracy_score
import ast

# Load Data
df = pd.read_csv('DataSet2_with_POS.csv')
df['tokens'] = df['tokens'].apply(ast.literal_eval)
df['tags'] = df['tags'].apply(ast.literal_eval)

def wordstofeatures(sent, i):
# Defining a function to extract features from a word in a sentence.
    word = sent[i] # Storing the word at position i in the sentence.
    features = {
        'bias': 1.0,
        'word.lower()': word.lower(),
        'word[-3:]': word[-3:], # The last three characters of the word.
        'word[-2:]': word[-2:], # The last two characters of the word.
        'word.isupper()': word.isupper(),
        'word.istitle()': word.istitle(),
        'word.isdigit()': word.isdigit(),
    }
    # Contextual features using neighboring words.
    if i > 0:
        features.update({
            '-1:word.lower()': sent[i-1].lower(), # The previous word in lowercase.
            '-1:word.istitle()': sent[i-1].istitle(),
        })
    if i < len(sent) - 1:
        features.update({
            '+1:word.lower()': sent[i+1].lower(),
            '+1:word.istitle()': sent[i+1].istitle(),
        })
    return features

def sent2features(sent):
    return [word2features(sent, i) for i in range(len(sent))]
# Applying feature extraction to each word in the sentence by using wordstofeatures function.
    # The function iterates over each word by its index,
    # extracts relevant features using the word2features function,
    # and compiles these into a list of feature dictionaries.

X = df['tokens'].apply(sent2features).tolist()
# Converting the tokens to features for all sentences in the dataframe and and converting it to list format for the CRF model.
y = df['tags'].tolist()

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)
# Splitting the dataset into training and testing sets using a 25% data for testing.

crf = CRF(algorithm='lbfgs', c1=0.1, c2=0.1, max_iterations=200, all_possible_transitions=True)
# Initializing the CRF model with the 'lbfgs' optimization algorithm and regularization parameters.

crf.fit(X_train, y_train)

y_pred = crf.predict(X_test)

report = classification_report(y_test, y_pred, output_dict=True)
# Generating a detailed classification report that includes precision, recall, F1-score for each tag.

print(" Classification Report:\n", classification_report(y_test, y_pred))
print("Precision: {:.2f}".format(report['weighted avg']['precision'])) # the format function rounds off the decimal points
print("Recall: {:.2f}".format(report['weighted avg']['recall']))
print("Accuracy: {:.2f}".format(accuracy_score(y_test, y_pred)))
print("F1-score: {:.2f}".format(report['weighted avg']['f1-score']))

 Classification Report:
               precision    recall  f1-score   support

    B-TVshow       0.48      0.25      0.33       393
   B-company       0.51      0.26      0.34       412
     B-event       0.55      0.31      0.40       556
     B-group       0.57      0.36      0.44       561
  B-location       0.68      0.43      0.53       329
    B-person       0.70      0.64      0.67      1185
   B-product       0.73      0.50      0.59       443
    I-TVShow       0.44      0.36      0.40       530
   I-company       0.21      0.12      0.15       137
     I-event       0.54      0.31      0.39       603
     I-group       0.21      0.17      0.19       196
  I-location       0.50      0.39      0.44       140
    I-person       0.70      0.64      0.67       398
   I-product       0.05      0.04      0.04       158
           O       0.92      0.98      0.95     31006

    accuracy                           0.88     37047
   macro avg       0.52      0.38      0.43     37047
w

In [ ]:
LSTM 

In [ ]:
#Training Dataset

In [11]:
import numpy as np
import pandas as pd
from tensorflow import keras
from tensorflow.keras.models import Model
from tensorflow.keras.layers import LSTM, Embedding, Dense, TimeDistributed, Input, Bidirectional
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score
import ast

# Load and preprocess the data
df = pd.read_csv('dataset1_with_POS.csv')
df['tokens'] = df['tokens'].apply(ast.literal_eval)
df['tags'] = df['tags'].apply(ast.literal_eval)

# Creating a list of all sentences and the corresponding tags
sentences = df['tokens'].tolist()
tags = df['tags'].tolist()

# Encoding words and tags
wordenc = LabelEncoder() # Using a label encoder for the tokens to convert words to integers for model processing.
tagenc = LabelEncoder()

allwords = [word for sent in sentences for word in sent]  # Combining all words from all sentences into a single list for the encoder.
alltags = [tag for taglist in tags for tag in taglist]

wordenc.fit(allwords) # Training the label encoder on the list of words to create a mapping from word to unique integer in encoder.
tagenc.fit(alltags)

# Transform each sentence and tag list to encoded form
X_enc = [wordenc.transform(sent) for sent in sentences] # Converting each sentence from a list of words to a list of integers based on the previously trained encoder.
y_enc = [tagenc.transform(taglist) for taglist in tags]

# Padding sequences
max_len = max(len(x) for x in X_enc) # Determining the maximum length of any sentence for padding
X_pad = pad_sequences(X_enc, maxlen=max_len, padding='post')
y_pad = pad_sequences(y_enc, maxlen=max_len, padding='post')

# Convert tag sequences to categorical
y_cat = [to_categorical(i, num_classes=len(tagenc.classes_)) for i in y_pad] # Convert each padded tag sequence into a one-hot encoded format for classification

# Splitting data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X_pad, np.array(y_cat), test_size=0.1, random_state=42)

# Build the model
input = Input(shape=(max_len,)) # Defining the input layer 
model = Embedding(input_dim=len(wordenc.classes_), output_dim=50)(input)  # Embedding layer to convert token indices to vectors.
model = Bidirectional(LSTM(units=64, return_sequences=True, recurrent_dropout=0.1))(model)# Bidirectional LSTM for learning from both directions of sequences.
out = TimeDistributed(Dense(len(tagenc.classes_), activation="softmax"))(model)# TimeDistributed Dense layer to predict a tag for each token.

model = Model(input, out)
model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])# Compiling the model with optimizer, loss function, and evaluation metrics.

# Training the model with 8 epochs
model.fit(X_train, y_train, batch_size=32, epochs=8, validation_split=0.1, verbose=1)

# Evaluating the model
eval = model.evaluate(X_test, y_test)
print(f'Loss: {eval[0]}, Accuracy: {eval[1]}')

# Prediction
y_pred = model.predict(X_test)

# Converting predictions and true labels to original format for evaluation
y_pred_argmax = np.argmax(y_pred, axis=-1).flatten()
y_test_argmax = np.argmax(y_test, axis=-1).flatten()

# Calculating precision, recall, and F1-score
accuracy = accuracy_score(y_test_argmax, y_pred_argmax)
precision = precision_score(y_test_argmax, y_pred_argmax, average='weighted',zero_division=0)
recall = recall_score(y_test_argmax, y_pred_argmax, average='weighted',zero_division=0)
f1 = f1_score(y_test_argmax, y_pred_argmax, average='weighted',zero_division=0)

print(f'Precision: {precision:.2f}')
print(f'Accuracy: {accuracy:.2f}')
print(f'Recall: {recall:.2f}')
print(f'F1 Score: {f1:.2f}')

Epoch 1/8
15/15 ━━━━━━━━━━━━━━━━━━━━ 2s 44ms/step - accuracy: 0.5470 - loss: 2.2112 - val_accuracy: 0.7322 - val_loss: 0.8738
Epoch 2/8
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step - accuracy: 0.8215 - loss: 0.7401 - val_accuracy: 0.9171 - val_loss: 0.4360
Epoch 3/8
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step - accuracy: 0.9184 - loss: 0.4385 - val_accuracy: 0.9222 - val_loss: 0.3950
Epoch 4/8
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step - accuracy: 0.9218 - loss: 0.4008 - val_accuracy: 0.9236 - val_loss: 0.3854
Epoch 5/8
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step - accuracy: 0.9235 - loss: 0.3885 - val_accuracy: 0.9239 - val_loss: 0.3815
Epoch 6/8
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step - accuracy: 0.9207 - loss: 0.3972 - val_accuracy: 0.9236 - val_loss: 0.3783
Epoch 7/8
15/15 ━━━━━━━━━━━━━━━━━━━━ 1s 36ms/step - accuracy: 0.9204 - loss: 0.3909 - val_accuracy: 0.9239 - val_loss: 0.3749
Epoch 8/8
15/15 ━━━━━━━━━━━━━━━━━━━━ 1s 37ms/step - accuracy: 0.9197 - loss: 0.3833 - val_accuracy: 0.9239 - val_loss:

In [ ]:
#Testing Dataset

In [7]:
import numpy as np
import pandas as pd
from tensorflow import keras
from tensorflow.keras.models import Model
from tensorflow.keras.layers import LSTM, Embedding, Dense, TimeDistributed, Input, Bidirectional
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score
import ast

# Load and preprocess the data
df = pd.read_csv('dataset2_with_POS.csv')
df['tokens'] = df['tokens'].apply(ast.literal_eval)
df['tags'] = df['tags'].apply(ast.literal_eval)

# Creating a list of all sentences and the corresponding tags
sentences = df['tokens'].tolist()
tags = df['tags'].tolist()

# Encoding words and tags
wordenc = LabelEncoder() # Using a label encoder for the tokens to convert words to integers for model processing.
tagenc = LabelEncoder()

allwords = [word for sent in sentences for word in sent]  # Combining all words from all sentences into a single list for the encoder.
alltags = [tag for taglist in tags for tag in taglist]

wordenc.fit(allwords) # Training the label encoder on the list of words to create a mapping from word to unique integer in encoder.
tagenc.fit(alltags)

# Transform each sentence and tag list to encoded form
X_enc = [wordenc.transform(sent) for sent in sentences] # Converting each sentence from a list of words to a list of integers based on the previously trained encoder.
y_enc = [tagenc.transform(taglist) for taglist in tags]

# Padding sequences
max_len = max(len(x) for x in X_enc) # Determining the maximum length of any sentence for padding
X_pad = pad_sequences(X_enc, maxlen=max_len, padding='post')
y_pad = pad_sequences(y_enc, maxlen=max_len, padding='post')

# Convert tag sequences to categorical
y_cat = [to_categorical(i, num_classes=len(tagenc.classes_)) for i in y_pad] # Convert each padded tag sequence into a one-hot encoded format for classification

# Splitting data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X_pad, np.array(y_cat), test_size=0.1, random_state=42)

# Build the model
input = Input(shape=(max_len,)) # Defining the input layer 
model = Embedding(input_dim=len(wordenc.classes_), output_dim=50)(input)  # Embedding layer to convert token indices to vectors.
model = Bidirectional(LSTM(units=64, return_sequences=True, recurrent_dropout=0.1))(model)# Bidirectional LSTM for learning from both directions of sequences.
out = TimeDistributed(Dense(len(tagenc.classes_), activation="softmax"))(model)# TimeDistributed Dense layer to predict a tag for each token.

model = Model(input, out)
model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])# Compiling the model with optimizer, loss function, and evaluation metrics.

# Training the model with 8 epochs
model.fit(X_train, y_train, batch_size=32, epochs=8, validation_split=0.1, verbose=1) # Training the model

# Evaluating the model
eval = model.evaluate(X_test, y_test)
print(f'Loss: {eval[0]}, Accuracy: {eval[1]}')

# Prediction
y_pred = model.predict(X_test)  # Print the loss and accuracy obtained on the test set.

# Converting predictions and true labels to original format for evaluation
y_pred_argmax = np.argmax(y_pred, axis=-1).flatten()
y_test_argmax = np.argmax(y_test, axis=-1).flatten()

# Calculating precision, recall, and F1-score
accuracy = accuracy_score(y_test_argmax, y_pred_argmax)
precision = precision_score(y_test_argmax, y_pred_argmax, average='weighted',zero_division=0)
recall = recall_score(y_test_argmax, y_pred_argmax, average='weighted',zero_division=0)
f1 = f1_score(y_test_argmax, y_pred_argmax, average='weighted',zero_division=0)

print(f'Precision: {precision:.2f}')
print(f'Accuracy: {accuracy:.2f}')
print(f'Recall: {recall:.2f}')
print(f'F1 Score: {f1:.2f}')

Epoch 1/8
72/72 ━━━━━━━━━━━━━━━━━━━━ 4s 28ms/step - accuracy: 0.7435 - loss: 1.2661 - val_accuracy: 0.9315 - val_loss: 0.3545
Epoch 2/8
72/72 ━━━━━━━━━━━━━━━━━━━━ 2s 35ms/step - accuracy: 0.9326 - loss: 0.3445 - val_accuracy: 0.9314 - val_loss: 0.3316
Epoch 3/8
72/72 ━━━━━━━━━━━━━━━━━━━━ 3s 40ms/step - accuracy: 0.9325 - loss: 0.3155 - val_accuracy: 0.9315 - val_loss: 0.2957
Epoch 4/8
72/72 ━━━━━━━━━━━━━━━━━━━━ 3s 40ms/step - accuracy: 0.9331 - loss: 0.2671 - val_accuracy: 0.9320 - val_loss: 0.2692
Epoch 5/8
72/72 ━━━━━━━━━━━━━━━━━━━━ 3s 41ms/step - accuracy: 0.9347 - loss: 0.2272 - val_accuracy: 0.9341 - val_loss: 0.2611
Epoch 6/8
72/72 ━━━━━━━━━━━━━━━━━━━━ 3s 40ms/step - accuracy: 0.9436 - loss: 0.1941 - val_accuracy: 0.9345 - val_loss: 0.2564
Epoch 7/8
72/72 ━━━━━━━━━━━━━━━━━━━━ 3s 40ms/step - accuracy: 0.9484 - loss: 0.1804 - val_accuracy: 0.9360 - val_loss: 0.2621
Epoch 8/8
72/72 ━━━━━━━━━━━━━━━━━━━━ 3s 40ms/step - accuracy: 0.9521 - loss: 0.1664 - val_accuracy: 0.9388 - val_loss: